In [1]:
import re
from util import read_passages
from operator import itemgetter, attrgetter

In [51]:
firstConvertion = {
    "goal": "OBJECTIVE",
    "hypothesis": "OBJECTIVE",
    "fact": "BACKGROUND",
    "problem": "BACKGROUND",
    "method": "METHODS",
    "result": "RESULTS",
    "implication": "CONCLUSIONS",
    "none":"BACKGROUND"
}

def hierarchicalDecision(labels):
    hierarchy = ["OBJECTIVE", "BACKGROUND", "METHODS", "RESULTS", "CONCLUSIONS"]
    for answer in hierarchy:
        if answer in labels:
            return answer
    assert(0)
    
def mergeFigureApprearance(figures):
    deduplicated = list(set(figures))
    if len(deduplicated)>1:
        no_nan = []
        for item in deduplicated:
            if type(item)==str:
                no_nan.extend(item.split("|"))
        deduplicated = ["|".join(list(set(no_nan)))]
        
    return deduplicated[0]

In [52]:
def clause2sentence(str_seqs, label_seqs):
    sentence_seqs = []
    new_label_seqs = []
    for str_seq, label_seq in zip(str_seqs, label_seqs):
        sentence_seq = []
        new_label_seq = []
        labels = []
        last_token = ""
        sentence = ""
        for clause, label in zip(str_seq, label_seq):
            if last_token == ".":
                sentence_seq.append(sentence)
                RCT_labels = [firstConvertion[label] for label in labels]
                new_label_seq.append(hierarchicalDecision(RCT_labels))
                #print(sentence, figure_appearance, mergeFigureApprearance(figure_appearance))
                sentence = ""
                labels = []

            clause = clause.strip()
            sentence+=clause+" "
            labels.append(label)
            tokens = clause.split(" ")
            last_token = tokens[-1]
            
        sentence_seq.append(sentence)
        RCT_labels = [firstConvertion[label] for label in labels]
        new_label_seq.append(hierarchicalDecision(RCT_labels))
        
        sentence_seqs.append(sentence_seq)
        new_label_seqs.append(new_label_seq)

    return sentence_seqs, new_label_seqs

In [56]:
str_seqs, label_seqs = read_passages("lucky_train.txt", True)

In [57]:
new_str_seqs, new_label_seqs = clause2sentence(str_seqs, label_seqs)

In [58]:
with open("lucky_train_sentence.txt","w") as f:
    for str_seq, label_seq in zip(new_str_seqs, new_label_seqs):
        for sentence, label in zip(str_seq, label_seq):
            f.write(sentence+"\t"+label+"\n")
        f.write("\n")